In [1]:
import json 
import xmltodict 
import os
import time
import datetime
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import re
from pymystem3 import Mystem
import pymorphy2
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer as rt
from bson.son import SON
import pprint

import pymongo
from pymongo import TEXT
from bson import ObjectId

# Предобработка данных

## Создание списка с XML

In [2]:
path_60 = '/Users/liliyarodicheva/Documents/GitHub/Elena_Shvarts/TEI/60'
path_ZT = '/Users/liliyarodicheva/Documents/GitHub/Elena_Shvarts/TEI/Zelenaya_tetrad'

In [3]:
def create_list_of_xml_from_dir(path): #функция, которая наши xml-файлы из директории читает и кладет в список
    shvarts_xml= []
    for i, file in enumerate(os.listdir(path)):
        if file.endswith ('.xml'):
            fullname = os.path.join(path, file)
            with open(fullname, encoding='utf8') as xml_file:
                xml = xml_file.read()
                xml = re.sub('/text>\n</TEI>', '/text>\n<ID>'+str(i)+'</ID><file>'+file+'</file></TEI>\n', xml)
                xml = re.sub('<fileDesc>\n', '<fileDesc>\n<root></root>', xml)
                shvarts_xml.append(xml)
                xml_file.close() 
        
    return shvarts_xml

In [4]:
#применяем нашу функцию по доставанию XML к директории, в которой лежат тексты 60-х годов
shvarts_60_xml = create_list_of_xml_from_dir(path_60)
shvarts_ZT_xml = create_list_of_xml_from_dir(path_ZT)

## Подключение к базе данных

In [5]:
'''Пока что начинаем с того, что дропаем нашу базу данных'''
client = pymongo.MongoClient('localhost', 27017) #подключаемся к MongoDB
db = client['admin'] #создаем курсор для конкретной базы/контейнера
db
db['Shvarts_60'].drop()#поскольку мы постоянно пробуем и терять нам нечего, дропаем базу прям сразу же
db['Shvarts_70'].drop()
db['Shvarts_ZT'].drop()

In [6]:
collection_60 = db["Shvarts_60"] #создаем внутри базы коллекции, где у нас лежат тексты/леммы/токены/строки за 60-е годы
collection_70 = db.Shvarts_70
collection_ZT = db.Shvarts_ZT

In [7]:
example = "<publisher>Сочинения, том 5. С.233 <date type='publishing'>2013</date></publisher>"
example = bs(example, 'xml')
example = example.find('publisher').next_element
# example = example.next_element
print(example)

Сочинения, том 5. С.233 


##   Получаем данные по xml в виде json/словаря (как в тетрадке), а заодно кладем все в базу (операция Create).

In [8]:
'''Создаем класс, как в примере с N+1 и все кладем в него, 
на выходе у нас данные не только в виде строк, но и в виде чисел'''

class Shvarts_poems: 
    def __init__(self):
        self.title=""
        self.root=""
        self.author=""
        self.editors=""
        self.publishers=""
        self.edition=""
        self.date_published=""
        self.date_written=""
        self.text=""
        self.ID=""
        self.file=""
        self.root=""
        self.children=""

def getPoemInfoShvarts(text):
    text = re.sub('publishing', 'published', text)
    text = re.sub('Издатели:', '', text)
    text = re.sub('Редакторы:', '', text)
    soup = bs(text, 'xml')
    
    poem=Shvarts_poems()
    
    poem.title = soup.find('title').text
    poem.author = soup.find('author').text
    
    
    poem.editors = ''
    poem.publishers = ''
    for i, resp in enumerate(soup.find_all('respStmt')):
        if i == 0:
            poem.editors += resp.text
        if i == 1:
            poem.publishers += resp.text       
    poem.editors = re.sub('\s+', ' ', poem.editors)
    poem.publishers = re.sub('\s+', ' ', poem.publishers)
    
    
    poem.edition = soup.find('publisher').next_element
    poem.edition = re.sub('\s+', ' ', poem.edition)
    
    
    poem.date_published = soup.find('date', {'type':'published'}).text
    if poem.date_published is not None:
        poem.date_published = int(poem.date_published)
     
    
    poem.date_written = soup.find('date', {'type':'written'}).text
    if poem.date_written in ('', ' '):
        poem.date_written = None
    elif poem.date_written is not None:
        poem.date_written = int(poem.date_written)
        
    poem.ID = soup.find('ID').text
    if poem.ID is not None:
        poem.ID = int(poem.ID)
        
    poem.file = soup.find('file').text
    
    poem.root = []
    poem.children = []

    poem.text = soup.find('lg').text
    
    return poem

In [9]:
'''также мы пробовали pymystem3, однако у него струкутра не такая удобная и тэги не такие понятные и лаконичные'''
morph=pymorphy2.MorphAnalyzer() 

In [10]:
def putShvartsPoemsInMongo(text):
    tokenizer = rt('\w+')
    # Загружаем текст стихотворения и прочие элементы
    poem = getPoemInfoShvarts(text) 
    
    full_text = {"meta": {"title": poem.title, "author": poem.author, 
                           "editors": poem.editors, "publishers":poem.publishers,
                           "edition":poem.edition, "date_published":poem.date_published,
                           "date_written":poem.date_written}, 
                "ID":poem.ID, "file":poem.file, "root":poem.root, "children":poem.children, 
                 "title": poem.title, "poem_text": poem.text}

    text_id = collection_60.insert_one(full_text).inserted_id

In [11]:
'''На этом моменте мы кладем в нашу бд все, что у нас имеется'''
db['Shvarts_60'].drop()
for text in shvarts_60_xml:
    putShvartsPoemsInMongo(text)
    
for text in collection_60.find({'title':{"$exists":True}}):
    if re.search('_\d+.xml', text['file']):
        changed_duplicate = re.sub('_\d+.xml', '.xml', text['file'])
        collection_60.update_one({'file':changed_duplicate},{'$push':{'children':text['_id']}})
        original_poem = collection_60.find({'file' : changed_duplicate}, 
                                                projection = {'file':True, '_id':True, 'children':True})[0]
        
        collection_60.update_one({'file':text['file']}, {'$push':{'root':original_poem['_id']}})
            
            
collection_60.create_index([('poem_text', 'text')], default_language='russian')

collection_60.count_documents({})

35

## Операция Read - читаем, что у нас в базе данных существует

In [12]:
def find_text(where, which):
    for text in collection_60.find({where:which}, projection={"_id":False, "poem_text": True}):
        print(text)

In [13]:
find_text('title','Юродивый')

In [14]:
duplicates = []
for text in collection_60.find({'root': {'$eq':'root'}}, projection={"_id":False, "poem_text": False}):
    duplicates.append(text)
print(duplicates)

[]


In [15]:
titles = [title for title in collection_60.find({"root": "root"}).sort('meta.date_written')]
titles

[]

In [16]:
file_names = []
roots = []
for i, element in enumerate(collection_60.find({"root": {"$exists":True}})):
    meta_for_compare = element['meta']
    file_names.append(meta_for_compare['file'])
    roots.append(element['root'])
    
for root in roots:
    for text in collection_60.find(projection = {'title' : True, 'meta.file': True, 'root':True, "_id":False}):
        if 'text.meta.file' == root:
            print(root)
        

KeyError: 'file'

## Операция Update - можем обновить наши данные

In [ ]:
'''Например мы видим, что в некоторых местах у нас указание даты написания равно None, а мы уже обрели знание,
что текст был написан тогда-то'''

collection_60.find_one_and_update({"meta.title":"Об изобретении паровой машины", "meta.date_written":'1967'},{"$set":{"meta.date_written":1967}}, upsert=False)

In [ ]:
for text in collection_60.find({'meta.title':'Об изобретении паровой машины'}):
    print(text)

In [ ]:
def update_data(collection, title, name, where, item, for_what):
    collection.find_one_and_update({title:name,
                                       where:item},
                                      {"$set":{where:for_what}}, upsert=False)
    
    for text in collection.find({title:name}):
        print(text)
    
update_data(collection_60, 'meta.title', 'Об изобретении паровой машины',
           'meta.date_written', None, 1967)

## Операция Delete

In [ ]:
'''Помним, что там где-то у нас завалялась Агния Львовна - давайте удалим ее, она ж не Шварц'''
print(collection_60.count_documents({}))

for text in collection_60.find({'TEI.teiHeader.title':'Мишка'}):
    print(text)

collection_60.delete_many({'TEI.teiHeader.title':'Мишка'})        

print(collection_60.count_documents({}))

## Sort, Regex, Aggregation, полнотекстовый поиск

* Sort

In [ ]:
for text in collection_60.find({'meta.date_written':{'$gte':1960}}, projection = {'title' : True, 'meta.date_written': True, "_id":False}).sort('meta.date_written'):
    print(text)
# for text in dictionary.find({"freq":{"$gt": 28}}, {"token": True, "freq":True, "_id":False}).sort("freq"):
#     print(text)

* RegExp

In [ ]:
def search_re(where, word):
    for text in collection_60.find({where:{'$regex':'.*'+word+'.*'}}, projection = {'_id':False, 'meta.title':True, 'poem_text':True}):
        print(text)    

In [ ]:
search_re('poem_text','дерев')

* Aggregation

In [ ]:
titles_pipeline = [{"$unwind": "$title"},
            {"$group": {"_id": "$title", "count": {"$sum": 1}}},
            {"$sort": SON([("count", -1), ("_id", -1)])}
           ]
pprint = (list(collection_60.aggregate(titles_pipeline)))
pprint

In [ ]:
titles_pipeline_1 = [{"$unwind": "$title"},
            {"$group": {"_id": "$title"}}
           ]
pprint = (list(collection_60.aggregate(titles_pipeline_1)))

titles = [title for title in collection_60.find({"title": {"$exists": True}}).sort('meta.date_written')]
titles = [title for title in collection_60.find({"root": None}).sort('meta.date_written')]
titles

In [ ]:
date_pipeline = [{"$unwind": "$meta.date_published"},
            {"$group": {"_id": "$meta.date_published", "count": {"$sum": 1}}},
            {"$sort": SON([("count", -1), ("_id", -1)])}
           ]
pprint.pprint(list(collection_60.aggregate(date_pipeline)))

* Full-text search

In [ ]:
for text in collection_60.find({"$text": {"$search": 'болотами'}}, projection={'_id':False}):
    print(text)

## Проверка кода для бэкэнда

In [ ]:
poem_texts = [text for text in collection_60.find({"poem_text": {"$exists": True}})]
for text in poem_text:

In [ ]:
def find_text():
    client = pymongo.MongoClient('mongodb://localhost:27017')
    db = client['admin']
    collection_60 = db.Shvarts_60
    texts = [text for text in collection_60.find({"poem_text": {"$exists": True}}, 
                                                 projection={"_id":False, "meta": False})]
    return texts

In [ ]:
find_text()

In [ ]:
def search(word):
    texts = [text for text in collection_60.find({"$text": {"$search": word}}, 
             projection={'_id':False, 'meta':False})]

#     poems = []

#     for text in texts:
#         lst = []
#         lst.append(text.get('ID'))
#         lst.append(text.get('title'))
#         poems.append(lst)

    return texts

In [ ]:
search('дерево')

In [ ]:
poem_texts = [text for text in collection_60.find({"poem_text": {"$exists": True}}).sort('meta.date_written')]

In [ ]:
poem_texts

In [ ]:
titles = [title for title in collection_60.find({"title": {"$exists": True}}).sort('meta.date_written')]
titles

In [ ]:
titles = [title for title in collection_60.find({"root": []}).sort('meta.date_written')]
print(titles)